# Covid-19 Dashboard

This Binder displays an interactive graph of the number of Covid-19 carried out daily, shown seperately for England, Scotland, Wales and Northern Ireland. The data was obtained through [Public Health England](https://www.gov.uk/government/organisations/public-health-england)'s [API](https://coronavirus.data.gov.uk/details/developers-guide) and visualised with the [pandas](https://pandas.pydata.org) data analysis library, and [Jupyter](https://jupyter.org/)'s [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/) library. The controls are easy to use:
* To change which nations are displayed, simply click and drag to highlight the desired nations, or [CTRL + click] each nation you want.
* The nations can be displayed on a single graph, or on seperate graphs, by toggling the Subplots option.
* To view the most up to date data, click the red button to refresh the graphs, redrawing them with any new data.

In [33]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time
from uk_covid19 import Cov19API

In [15]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [2]:
# Load JSON files and store the raw data in some variable.
with open("testsByNation.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [3]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def create_date_range(data):
    # generate a date range from a json datafile containing date strings
    
    datalist = data['data']
    
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    return pd.date_range(startdate, enddate, freq='D')

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe. """
    datalist = rawdata['data']
    
    nationlist = []
    for entry in datalist: # each entry is a dictionary with date, nation and new tests
        if entry['nation'] not in nationlist:
            nationlist.append(entry['nation'])

    index=create_date_range(rawdata)
    myTimeseriesdf=pd.DataFrame(index=index, columns=nationlist)

    for entry in datalist:
        date=parse_date(entry['date'])
        column = entry['nation']
        # check that nothing is there yet - just in case some dates are duplicated,
        # maybe with data for different columns in each entry
        if pd.isna(myTimeseriesdf.loc[date, column]): 
            # replace None with 0 in our data 
            value = float(entry['tests']) if entry['tests']!=None else 0.0
            myTimeseriesdf.loc[date, column] = value

    # fill in any remaining "holes" due to missing dates
    myTimeseriesdf.fillna(0.0, inplace=True)
    
    return myTimeseriesdf

df=wrangle_data(jsondata) # df is the dataframe for plotting

In [4]:
def access_api():
    # Access the covid API and returns the tests data, with the latest data.
    filters = [
        'areaType=nation',
    ]

    structure = {
        "date": "date",
        "nation": "areaName",
        "tests": "newTestsByPublishDate"
    }

    api = Cov19API(filters=filters, structure=structure)
    testsByNation = api.get_json()

    return testsByNation # return data read from the API

In [38]:
def api_button_callback(button):
    """ Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    refresh_graph()
    apibutton.icon="check-circle"
    time.sleep(2) # delay to allow the user to acknowledge the successful refresh, then resets the button
    apibutton.icon="download"
    

    
apibutton=wdg.Button(
    description='Refresh',
    button_style='danger',
    tooltip="Click to refresh data from Public Health England",
    icon='download'
)

apibutton.on_click(api_button_callback)

#display(apibutton)

In [50]:
nationcols=wdg.SelectMultiple(
    options=['England', 'Northern Ireland', 'Scotland', 'Wales'],
    value=['England', 'Northern Ireland', 'Scotland', 'Wales'], # initial value
    rows=4,
    description='Nations',
    disabled=False
)

subplot_btn=wdg.RadioButtons(
    options=['True', 'False'],
    value='False', # Defaults to false
    description='Subplots:',
)

date_range = create_date_range(jsondata)
options = [(date.strftime(' %d/%m/%Y '), date) for date in date_range]
timeframewdg = wdg.SelectionRangeSlider(
    options = options,
    index = (0,len(options)-1),
    description='Dates',
    layout={'width': '500px'}
)

def tests_graph(graphcolumns, subplot, timeframe):
    # the callback function.
    if subplot == 'True':
        subp_value=True
    else:
        subp_value=False
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(subplots=subp_value, figsize=(15,7), xlabel="Date", ylabel="Tests", y=list(graphcolumns)) # graphcolumns is a tuple - we need a list
    else:
        # if the user has not selected any column, print a message instead
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
controls = wdg.HBox([nationcols, subplot_btn])

output=wdg.interactive_output(tests_graph, {'graphcolumns': nationcols, 'subplot': subplot_btn, 'timeframe': timeframewdg})

display(controls, output, apibutton)

#print(timeframewdg.value)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=nationcols.value
    nationcols.value=(nationcols.options[0],) # forces the redraw
    nationcols.value=current


Output()

Button(button_style='danger', description='Refresh', icon='download', style=ButtonStyle(), tooltip='Click to r…

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*